In [23]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data
df = px.data.tips()

df2 = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})
fig = px.bar(df2, x="Fruit", y="Amount", color="City", barmode="group")
# Build App
app = JupyterDash(__name__)
app.layout = html.Div(children=[
    html.Div([
        html.H1("JupyterDash Demo"),
        dcc.Graph(id='graph'),
        html.Label([
            "colorscale",
            dcc.Dropdown(
                id='colorscale-dropdown', clearable=False,
                value='plasma', options=[
                    {'label': c, 'value': c}
                    for c in px.colors.named_colorscales()
                ])
        ]),
    ]),
    html.Div(children=[
        html.H1(children='Hello Dash'),

        html.Div(children='''
        Dash: A web application framework for your data.
    '''),

        dcc.Graph(
            id='example-graph',
            figure=fig
        )
    ])
])
# Define callback to update graph


@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )


# Run app and display result inline in the notebook
app.run_server(mode='external')


Dash app running on http://127.0.0.1:8050/


## Vis3. Bubble Plot

<img src="../imgs/vis3_example.png" alt="drawing" width="500">

Data format

```
Index
0   tx: a single transaction

Column
1   tx_id
2   date
3   price
4   bubble_size: the attribute count of the punk
5   bubble_color: decided by the skin tone of the punk
6   from: trader_id
7   to: trader_id
8   punk_id
9   punk_img: the image directory of the punk
```

### Categorical Bubble Charts

In [20]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import math

In [22]:
data = px.data.gapminder()
df_2007 = data[data['year']==2007]
df_2007 = df_2007.sort_values(['continent', 'country'])
df_2007

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
35,Algeria,Africa,2007,72.301,33333216,6223.367465,DZA,12
47,Angola,Africa,2007,42.731,12420476,4797.231267,AGO,24
131,Benin,Africa,2007,56.728,8078314,1441.284873,BEN,204
167,Botswana,Africa,2007,50.728,1639131,12569.851770,BWA,72
203,Burkina Faso,Africa,2007,52.295,14326203,1217.032994,BFA,854
...,...,...,...,...,...,...,...,...
1487,Switzerland,Europe,2007,81.701,7554661,37506.419070,CHE,756
1583,Turkey,Europe,2007,71.777,71158647,8458.276384,TUR,792
1607,United Kingdom,Europe,2007,79.425,60776238,33203.261280,GBR,826
71,Australia,Oceania,2007,81.235,20434176,34435.367440,AUS,36


In [18]:

hover_text = []
bubble_size = []

for index, row in df_2007.iterrows():
    hover_text.append(('Country: {country}<br>'+
                      'Life Expectancy: {lifeExp}<br>'+
                      'GDP per capita: {gdp}<br>'+
                      'Population: {pop}<br>'+
                      'Year: {year}').format(country=row['country'],
                                            lifeExp=row['lifeExp'],
                                            gdp=row['gdpPercap'],
                                            pop=row['pop'],
                                            year=row['year']))
    bubble_size.append(math.sqrt(row['pop']))

df_2007['text'] = hover_text
df_2007['size'] = bubble_size
sizeref = 2.*max(df_2007['size'])/(100**2)

# Dictionary with dataframes for each continent
continent_names = ['Africa', 'Americas', 'Asia', 'Europe', 'Oceania']
continent_data = {continent:df_2007.query("continent == '%s'" %continent)
                              for continent in continent_names}

# Create figure
fig = go.Figure()

for continent_name, continent in continent_data.items():
    fig.add_trace(go.Scatter(
        x=continent['gdpPercap'], y=continent['lifeExp'],
        name=continent_name, text=continent['text'],
        marker_size=continent['size'],
        ))

# Tune marker appearance and layout
fig.update_traces(mode='markers', marker=dict(sizemode='area',
                                              sizeref=sizeref, line_width=2))

fig.update_layout(
    title='Life Expectancy v. Per Capita GDP, 2007',
    xaxis=dict(
        title='GDP per capita (2000 dollars)',
        gridcolor='white',
        type='log',
        gridwidth=2,
    ),
    yaxis=dict(
        title='Life Expectancy (years)',
        gridcolor='white',
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)
fig.show()

In [25]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [26]:
from jupyter_dash import JupyterDash
from dash import no_update

In [28]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
from dash import no_update

import plotly.graph_objects as go
import pandas as pd

# Small molcule drugbank dataset
# Source: https://raw.githubusercontent.com/plotly/dash-sample-apps/main/apps/dash-drug-discovery/data/small_molecule_drugbank.csv'
data_path = 'datasets/small_molecule_drugbank.csv'

df = pd.read_csv(data_path, header=0, index_col=0)

fig = go.Figure(data=[
    go.Scatter(
        x=df["LOGP"],
        y=df["PKA"],
        mode="markers",
        marker=dict(
            colorscale='viridis',
            color=df["MW"],
            size=df["MW"],
            colorbar={"title": "Molecular<br>Weight"},
            line={"color": "#444"},
            reversescale=True,
            sizeref=45,
            sizemode="diameter",
            opacity=0.8,
        )
    )
])

# turn off native plotly.js hover effects - make sure to use
# hoverinfo="none" rather than "skip" which also halts events.
fig.update_traces(hoverinfo="none", hovertemplate=None)

fig.update_layout(
    xaxis=dict(title='Log P'),
    yaxis=dict(title='pkA'),
    plot_bgcolor='rgba(255,255,255,0.1)'
)

app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Graph(id="graph-basic-2", figure=fig, clear_on_unhover=True),
    dcc.Tooltip(id="graph-tooltip"),
])


@app.callback(
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("graph-basic-2", "hoverData"),
)

def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    # demo only shows the first point, but other points may also be available
    pt = hoverData["points"][0]
    bbox = pt["bbox"]
    num = pt["pointNumber"]

    df_row = df.iloc[num]
    img_src = df_row['IMG_URL']
    name = df_row['NAME']
    form = df_row['FORM']
    desc = df_row['DESC']
    if len(desc) > 300:
        desc = desc[:100] + '...'

    children = [
        html.Div([
            html.Img(src=img_src, style={"width": "100%"}),
            html.H2(f"{name}", style={"color": "darkblue"}),
            html.P(f"{form}"),
            html.P(f"{desc}"),
        ], style={'width': '200px', 'white-space': 'normal'})
    ]

    return True, bbox, children

app.run_server(mode='external')


Dash app running on http://127.0.0.1:8050/


In [29]:
df

,NAME,PAGE,IMG_URL,LOGP,PKA,MW,FORM,SOL,DESC
Unnamed: 0,,,,,,,,,
0,Cyclacillin,https://www.drugbank.ca/drugs/DB01000,https://www.drugbank.ca/structures/DB01000/thu...,1.31,3.30,341.4260,C15H23N3O4S,1.90000,A cyclohexylamido analog of penicillanic acid....
1,Salbutamol,https://www.drugbank.ca/drugs/DB01001,https://www.drugbank.ca/structures/DB01001/thu...,1.40,10.12,239.3107,C13H21NO3,2.15000,"Salbutamol is a short-acting, selective beta2-..."
2,Levobupivacaine,https://www.drugbank.ca/drugs/DB01002,https://www.drugbank.ca/structures/DB01002/thu...,3.60,13.62,288.4277,C18H28N2O,0.09770,Levobupivacaine is an amino-amide local anaest...
3,Cromoglicic acid,https://www.drugbank.ca/drugs/DB01003,https://www.drugbank.ca/structures/DB01003/thu...,1.92,1.77,468.3665,C23H16O11,0.03580,A chromone complex that acts by inhibiting the...
4,Ganciclovir,https://www.drugbank.ca/drugs/DB01004,https://www.drugbank.ca/structures/DB01004/thu...,-1.66,10.16,255.2306,C9H13N5O4,11.50000,An acyclovir analog that is a potent inhibitor...
...,...,...,...,...,...,...,...,...,...
685,N-(5&#39;-Phosphopyridoxyl)-D-Alanine,https://www.drugbank.ca/drugs/DB01993,https://www.drugbank.ca/structures/DB01993/thu...,-1.70,1.03,320.2356,C11H17N2O7P,1.48000,Not Available
686,"2-(Pyrido[1,2-E]Purin-4-Yl)Amino-Ethanol",https://www.drugbank.ca/drugs/DB01994,https://www.drugbank.ca/structures/DB01994/thu...,0.85,15.33,229.2379,C11H11N5O,0.19900,Not Available
687,5-Methylcytidine-5&#39;-Monophosphate,https://www.drugbank.ca/drugs/DB01995,https://www.drugbank.ca/structures/DB01995/thu...,-1.80,1.24,337.2231,C10H16N3O8P,15.30000,Not Available
